# Import packages

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

# Load data

In [3]:
sub = pd.read_csv('./data/raw/submission_history.csv')

In [4]:
sub.head()

,id,pk,track_id,track_info,band_id,influencer_id,influencer_kind,influencer_feedback,decision,score
0,7312,7312,324,test tim,303,102,Label,"Bonjour, \nle track surf sur les codes ""austra...",['give feedback on your tune'],0.0
1,7313,7313,324,test tim,303,103,Radio,"Bonjour, merci pour votre envoi. Le morceau n'...",['give feedback on your tune'],0.0
2,7314,7314,324,test tim,303,104,Journalist,Le morceau est à lui tout seul une succession ...,['give feedback on your tune'],0.0
3,7315,7315,324,test tim,303,105,Channel,Très bonne pop aux airs de Tame Impala et Pond...,"['share it on social media', 'add it to a play...",1.0
4,7316,7316,324,test tim,303,106,Media,"La production est assurément excellente, mais ...",['give feedback on your tune'],0.0


# Split data

In [5]:
N_FOLDS = 5
RANDOM_SEED = 42

In [6]:
X = sub.drop(columns=['score', 'influencer_feedback', 'decision'])
y = sub.score
skf = StratifiedKFold(n_splits=N_FOLDS, random_state=RANDOM_SEED)

/home/pa/.virtualenvs/groover/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [7]:
skf.get_n_splits(X, y)
print(skf)

StratifiedKFold(n_splits=5, random_state=42, shuffle=False)


# Naive models

In [8]:
from sklearn.metrics import mean_squared_error

In [9]:
std_train_mean = 0.0
std_test_mean = 0.0
naive_score = 0.0
random_score =  0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    std_train_mean += np.std(y_train)
    std_test_mean += np.std(y_test)
    naive_score += np.sqrt(mean_squared_error([y_train.mean()] * len(y_test), y_test))
    random_score += np.sqrt(mean_squared_error(np.random.uniform(0, 1, size=len(y_test)), y_test))

std_train_mean /= N_FOLDS
std_test_mean /= N_FOLDS
naive_score /= N_FOLDS
random_score /= N_FOLDS

In [10]:
print(std_train_mean)
print(std_test_mean)
print(naive_score)
print(random_score)

0.39570093979500714
0.39570093963773195
0.39570093998996175
0.5498874076131429


# LGBM model

In [32]:
from lightgbm import LGBMRegressor

In [11]:
band = pd.read_csv('./data/raw/band_content.csv')
content = pd.read_csv('./data/raw/influencer_content.csv')

In [12]:
sub.head()

,id,pk,track_id,track_info,band_id,influencer_id,influencer_kind,influencer_feedback,decision,score
0,7312,7312,324,test tim,303,102,Label,"Bonjour, \nle track surf sur les codes ""austra...",['give feedback on your tune'],0.0
1,7313,7313,324,test tim,303,103,Radio,"Bonjour, merci pour votre envoi. Le morceau n'...",['give feedback on your tune'],0.0
2,7314,7314,324,test tim,303,104,Journalist,Le morceau est à lui tout seul une succession ...,['give feedback on your tune'],0.0
3,7315,7315,324,test tim,303,105,Channel,Très bonne pop aux airs de Tame Impala et Pond...,"['share it on social media', 'add it to a play...",1.0
4,7316,7316,324,test tim,303,106,Media,"La production est assurément excellente, mais ...",['give feedback on your tune'],0.0


In [13]:
content.head()

,id,influencer_id,description_fr,description_en,preferences_fr,preferences_en,Acid house,African music,Alternative rock,Ambient,...,Singer-songwriter,Soul,Surf rock,Synthpop,Synthwave,Techno,Traditional Music,Trap,Trip hop,Variété Française
0,96,96,"Ex-BSC NEWS, nouveau magazine culturel franc-t...","Ex-BSC NEWS, nouveau magazine culturel franc-t...",Musique Comtemporaine et Jazz,NaN,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,97,97,Underdog Records is a french alternative label...,Underdog Records is a french alternative label...,"Folk, soul, blues, rock&roll, indie pop","Folk, soul, blues, rock&roll, indie pop",0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
2,102,102,HIGHLIFE is a music publishing company + Indep...,HIGHLIFE Recordings has a wide open philosophy...,Déjà de la maturité,Already mature and original,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
3,103,103,Nectar est une émission radio musicale et hebd...,Nectar is a weekly music radio program about f...,Folk,Folk,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,104,104,Ecrit pour Konbini et Noisey (Vice). Défricheu...,"Writes for Konbini and Noisey (Vice). Rap, Hip...",Rap,Rap,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1


In [14]:
dataset = sub[['id', 'track_id', 'band_id', 'influencer_id', 'influencer_kind', 'score']].merge(
    band.drop(columns=['id', 'biography_fr', 'biography_en']),
    how='left',
    on='band_id'
).merge(
    content.drop(columns=['id', 'description_fr', 'description_en', 'preferences_fr', 'preferences_en']),
    how='left',
    on='influencer_id'
)

In [15]:
dataset.columns.tolist()

['id',
 'track_id',
 'band_id',
 'influencer_id',
 'influencer_kind',
 'score',
 'Acid house_x',
 'African music_x',
 'Alternative rock_x',
 'Ambient_x',
 'Blues_x',
 'Bossa Nova_x',
 'Chill-out_x',
 'Classical Music_x',
 'Coldwave_x',
 'Country_x',
 'Dance music_x',
 'Dance-pop_x',
 'Deep house_x',
 'Disco_x',
 'Dream Pop_x',
 'Dub_x',
 'Electro swing_x',
 'Electronic rock_x',
 'Electronica_x',
 'Electropop_x',
 'Experimental_x',
 'Experimental Jazz_x',
 'Experimental rock_x',
 'Film Music_x',
 'French house_x',
 'Funk_x',
 'Future house_x',
 'Garage rock_x',
 'Grime_x',
 'Hard rock_x',
 'Hip hop_x',
 'House music_x',
 'Indie folk_x',
 'Indie pop_x',
 'Indie rock_x',
 'Instrumental_x',
 'International Pop_x',
 'Latin music_x',
 'Lo-Fi_x',
 'Metal_x',
 'Minimal_x',
 'Modern Jazz_x',
 'New wave_x',
 'Noise rock_x',
 'Nouvelle Scène_x',
 'Nu-disco_x',
 'Pop rock_x',
 'Pop soul_x',
 'Post-punk_x',
 'Post-rock_x',
 'Progressive pop_x',
 'Progressive rock_x',
 'Psychedelic pop_x',
 'Psyched

In [16]:
dataset.influencer_kind.value_counts()

Media          35288
Radio           9872
Label           9155
Playlist        8501
Journalist      4268
Channel         4150
Booker          3405
Mentor          2288
Manager         2094
Springboard     2087
Publisher       1698
Supervisor       572
Event            328
Name: influencer_kind, dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['influencer_kind'] = le.fit_transform(dataset['influencer_kind'])

In [18]:
dataset.head()

,id,track_id,band_id,influencer_id,influencer_kind,score,Acid house_x,African music_x,Alternative rock_x,Ambient_x,...,Singer-songwriter_y,Soul_y,Surf rock_y,Synthpop_y,Synthwave_y,Techno_y,Traditional Music_y,Trap_y,Trip hop_y,Variété Française_y
0,7312,324,303,102,4,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,7313,324,303,103,10,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7314,324,303,104,3,0.0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
3,7315,324,303,105,1,1.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,7316,324,303,106,6,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [19]:
X, y = (
    dataset.drop(columns='score').rename(
        {'Variété Française_x': 'Variete Francaise_x', 'Variété Française_y': 'Variete Francaise_y'},
        axis=1
    ).values,
    dataset.score.values
)

In [160]:
lgbm_score =  0.0


for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    
    lgbm = LGBMRegressor(num_leaves=2**9, learning_rate=0.5, n_estimators=1000, silent=False)
    lgbm.fit(X_tr, y_tr, eval_set=(X_valid, y_valid), 
             eval_metric='mse', early_stopping_rounds=100,
             verbose=20)
    
    lgbm_score += np.sqrt(mean_squared_error(lgbm.predict(X_test), y_test))

lgbm_score /= N_FOLDS

print(lgbm_score)

/home/pa/.virtualenvs/groover/lib/python3.6/site-packages/pandas/core/series.py:1146: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Training until validation scores don't improve for 100 rounds
[20]	valid_0's l2: 0.274816
[40]	valid_0's l2: 0.287167
[60]	valid_0's l2: 0.29211
[80]	valid_0's l2: 0.295347
[100]	valid_0's l2: 0.298137
Early stopping, best iteration is:
[1]	valid_0's l2: 0.219449


/home/pa/.virtualenvs/groover/lib/python3.6/site-packages/pandas/core/series.py:1146: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Training until validation scores don't improve for 100 rounds
[20]	valid_0's l2: 0.269737
[40]	valid_0's l2: 0.279144
[60]	valid_0's l2: 0.284053
[80]	valid_0's l2: 0.28695
[100]	valid_0's l2: 0.288752
Early stopping, best iteration is:
[1]	valid_0's l2: 0.209829


/home/pa/.virtualenvs/groover/lib/python3.6/site-packages/pandas/core/series.py:1146: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Training until validation scores don't improve for 100 rounds
[20]	valid_0's l2: 0.227908
[40]	valid_0's l2: 0.237535
[60]	valid_0's l2: 0.241986
[80]	valid_0's l2: 0.245577
[100]	valid_0's l2: 0.247749
Early stopping, best iteration is:
[1]	valid_0's l2: 0.192493


/home/pa/.virtualenvs/groover/lib/python3.6/site-packages/pandas/core/series.py:1146: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Training until validation scores don't improve for 100 rounds
[20]	valid_0's l2: 0.242655
[40]	valid_0's l2: 0.250859
[60]	valid_0's l2: 0.253517
[80]	valid_0's l2: 0.2564
[100]	valid_0's l2: 0.258503
Early stopping, best iteration is:
[1]	valid_0's l2: 0.192406


/home/pa/.virtualenvs/groover/lib/python3.6/site-packages/pandas/core/series.py:1146: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Training until validation scores don't improve for 100 rounds
[20]	valid_0's l2: 0.196023
[40]	valid_0's l2: 0.196839
[60]	valid_0's l2: 0.197512
[80]	valid_0's l2: 0.197899
[100]	valid_0's l2: 0.198361
Early stopping, best iteration is:
[1]	valid_0's l2: 0.17284
0.311422090874112


# First Keras models

In [150]:
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Dropout
from keras.models import Model

In [ ]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)prod = Dot(name="Dot-Product", axes=1)([book_vec, user_vec])
model = Model([user_input, book_input], prod)
model.compile('adam', 'mean_squared_error')

In [22]:
dataset.head()

,id,track_id,band_id,influencer_id,influencer_kind,score,Acid house_x,African music_x,Alternative rock_x,Ambient_x,...,Singer-songwriter_y,Soul_y,Surf rock_y,Synthpop_y,Synthwave_y,Techno_y,Traditional Music_y,Trap_y,Trip hop_y,Variété Française_y
0,7312,324,303,102,4,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,7313,324,303,103,10,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7314,324,303,104,3,0.0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
3,7315,324,303,105,1,1.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,7316,324,303,106,6,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [35]:
i_data = dataset.filter(regex='_x')
b_data = dataset.filter(regex='_y')

In [71]:
def build_model(i_emb_dim, b_emb_dim):
    """
    """
    influencer_input = Input(shape=[i_data.shape[1]], name="Influencer-Input")
    influencer_embedding = Dense(50, activation='tanh', name="Influencer-Embedding")(influencer_input)
    
    band_input = Input(shape=[b_data.shape[1]], name="Band-Input")
    band_embedding = Dense(50, activation='tanh', name="Band-Embedding")(band_input)
    
    prod = Dot(name="Dot-Product", axes=1)([influencer_embedding, band_embedding])
    model = Model([influencer_input, band_input], prod)
    model.compile('adam', 'mean_squared_error')
    
    return model

In [37]:
model = build_model()

In [38]:
history = model.fit([i_data, b_data], dataset.score, epochs=10, verbose=1)



Epoch 1/10
83706/83706 [==============================] - 6s 73us/step - loss: 0.1854
Epoch 2/10
83706/83706 [==============================] - 4s 51us/step - loss: 0.1366
Epoch 3/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1264
Epoch 4/10
83706/83706 [==============================] - 4s 51us/step - loss: 0.1195
Epoch 5/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1158
Epoch 6/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1131
Epoch 7/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1099
Epoch 8/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1086
Epoch 9/10
83706/83706 [==============================] - 4s 53us/step - loss: 0.1067
Epoch 10/10
83706/83706 [==============================] - 5s 54us/step - loss: 0.1059


In [39]:
# model.save('regression_model.h5')

In [50]:
X, y = (
    dataset.drop(columns='score').rename(
        {'Variété Française_x': 'Variete Francaise_x', 'Variété Française_y': 'Variete Francaise_y'},
        axis=1
    ).values,
    dataset.score.values
)

In [35]:
i_data = dataset.filter(regex='_x')
b_data = dataset.filter(regex='_y')

In [67]:
i_data_idx = [dataset.drop(columns='score').columns.get_loc(c) for c in dataset.filter(regex='_x')]
b_data_idx = [dataset.drop(columns='score').columns.get_loc(c) for c in dataset.filter(regex='_y')]

In [68]:
dataset.columns.get_loc('Variété Française_y')

147

In [69]:
dataset.shape

(83706, 148)

In [70]:
nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    
    model = build_model()
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx]], y_train,
              validation_data=([X_valid[:, i_data_idx], X_valid[:, b_data_idx]], y_valid), 
              batch_size=64,
              epochs=10,
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx]]), y_test))

nn_score /= N_FOLDS

print(nn_score)

Train on 46874 samples, validate on 20090 samples
Epoch 1/10
46874/46874 [==============================] - 2s 43us/step - loss: 0.2356 - val_loss: 0.1667
Epoch 2/10
46874/46874 [==============================] - 2s 34us/step - loss: 0.1518 - val_loss: 0.1449
Epoch 3/10
46874/46874 [==============================] - 2s 34us/step - loss: 0.1356 - val_loss: 0.1347
Epoch 4/10
46874/46874 [==============================] - 2s 34us/step - loss: 0.1276 - val_loss: 0.1331
Epoch 5/10
46874/46874 [==============================] - 2s 34us/step - loss: 0.1219 - val_loss: 0.1313
Epoch 6/10
46874/46874 [==============================] - 2s 35us/step - loss: 0.1170 - val_loss: 0.1236
Epoch 7/10
46874/46874 [==============================] - 2s 34us/step - loss: 0.1145 - val_loss: 0.1162
Epoch 8/10
46874/46874 [==============================] - 2s 34us/step - loss: 0.1118 - val_loss: 0.1195
Epoch 9/10
46874/46874 [==============================] - 2s 34us/step - loss: 0.1100 - val_loss: 0.1174
Epoch

In [73]:
dataset.influencer_kind.nunique()

13

In [151]:
def build_model_2(i_emb_dim=50, b_emb_dim=50, kind_emb_dim=5, last_dense=20, dropout=0.2):
    """
    """
    # Influencer embedding
    influencer_input = Input(shape=[i_data.shape[1]], name="Influencer-Input")
    influencer_embedding = Dense(i_emb_dim, activation='tanh', name="Influencer-Embedding")(influencer_input)
    
    # Influencer kind categorical embedding
    influencer_kind_input = Input(shape=[1], name="Influencer-Kind-Input")
    influencer_kind_emb = Embedding(14, kind_emb_dim, name="Influencer-Kind-Embedding")(influencer_kind_input)
    
    # Concatenate influencer emb with influencer kind emb to get full influencer emb
    influencer_full_emb = Concatenate(axis=-1)([influencer_embedding, Flatten(name='Flatten')(influencer_kind_emb)])
    
    # Band embedding
    band_input = Input(shape=[b_data.shape[1]], name="Band-Input")
    band_embedding = Dense(b_emb_dim, activation='tanh', name="Band-Embedding")(band_input)
    
    # Concatenate and create product
    prod = Concatenate(name="Concat", axis=-1)([influencer_full_emb, band_embedding])
    prod2 = Dense(last_dense, activation='tanh', name="Dense1")(prod)
    dropout = Dropout(rate=dropout)(prod2)
    
    # Dropout
    prod3 = Dense(1, activation='tanh', name="Dense2")(dropout)
    model = Model([influencer_input, band_input, influencer_kind_input], prod3)
    model.compile('adam', 'mean_squared_error')
    
    return model

In [181]:
def build_model_3(i_emb_dim=50, b_emb_dim=50, kind_emb_dim=10, last_dense=40, dropout=0.5):
    """
    """
    # Influencer embedding
    influencer_input = Input(shape=[i_data.shape[1]], name="Influencer-Input")
    influencer_embedding = Dense(i_emb_dim, activation='tanh', name="Influencer-Embedding1")(influencer_input)
    influencer_embedding = Dense(last_dense, activation='tanh', name="Dense1")(influencer_embedding)
    influencer_embedding = Dense(i_emb_dim-10, activation='tanh', name="Influencer-Embedding")(influencer_embedding)
    
    # Influencer kind categorical embedding
    influencer_kind_input = Input(shape=[1], name="Influencer-Kind-Input")
    influencer_kind_emb = Embedding(14, kind_emb_dim, name="Influencer-Kind-Embedding")(influencer_kind_input)
    
    # Concatenate influencer emb with influencer kind emb to get full influencer emb
    influencer_full_emb = Concatenate(axis=-1)([influencer_embedding, Flatten(name='Flatten')(influencer_kind_emb)])
    
    # Band embedding
    band_input = Input(shape=[b_data.shape[1]], name="Band-Input")
    band_embedding = Dense(b_emb_dim, activation='tanh', name="Band-Embedding1")(band_input)
    band_embedding = Dense(last_dense, activation='tanh', name="Dense2")(band_embedding)
    band_embedding = Dense(b_emb_dim-10, activation='tanh', name="Band-Embedding")(band_embedding)
    
    # Concatenate and create product
    prod = Concatenate(name="Concat", axis=-1)([influencer_full_emb, band_embedding])
    prod2 = Dense(last_dense, activation='tanh', name="Dense0")(prod)
    dropout = Dropout(rate=dropout)(prod2)
    
    # Dropout
    prod3 = Dense(1, activation='tanh', name="Dense3")(dropout)
    model = Model([influencer_input, band_input, influencer_kind_input], prod3)
    model.compile('adam', 'mean_squared_error')
    
    return model

In [182]:
dataset.head()

,id,track_id,band_id,influencer_id,influencer_kind,score,Acid house_x,African music_x,Alternative rock_x,Ambient_x,...,Singer-songwriter_y,Soul_y,Surf rock_y,Synthpop_y,Synthwave_y,Techno_y,Traditional Music_y,Trap_y,Trip hop_y,Variété Française_y
0,7312,324,303,102,4,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,7313,324,303,103,10,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7314,324,303,104,3,0.0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
3,7315,324,303,105,1,1.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,7316,324,303,106,6,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [183]:
EPOCHS = 100
PATIENCE = 10
BATCH_SIZE = 32

In [ ]:
nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    
    # Build model
    model = build_model_3()
    
    # Early stoppnig callback
    es = keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    
    # Fit
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx], X_train[:, 5]], y_train,
              validation_data=([X_valid[:, i_data_idx], X_valid[:, b_data_idx], X_valid[:, 5]], y_valid), 
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              callbacks=[es],
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(
        model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx], X_test[:, 5]]), y_test
    ))

nn_score /= N_FOLDS

print(nn_score)

Train on 46874 samples, validate on 20090 samples
Epoch 1/100
46874/46874 [==============================] - 11s 233us/step - loss: 0.1455 - val_loss: 0.1210
Epoch 2/100
46874/46874 [==============================] - 8s 173us/step - loss: 0.1163 - val_loss: 0.1066
Epoch 3/100
46874/46874 [==============================] - 8s 181us/step - loss: 0.1088 - val_loss: 0.1011
Epoch 4/100
46874/46874 [==============================] - 8s 169us/step - loss: 0.1052 - val_loss: 0.0999
Epoch 5/100
46874/46874 [==============================] - 8s 167us/step - loss: 0.1034 - val_loss: 0.1031
Epoch 6/100
46874/46874 [==============================] - 8s 177us/step - loss: 0.1019 - val_loss: 0.0984
Epoch 7/100
46874/46874 [==============================] - 8s 176us/step - loss: 0.1009 - val_loss: 0.0963
Epoch 8/100
46874/46874 [==============================] - 8s 174us/step - loss: 0.1006 - val_loss: 0.0991
Epoch 9/100
46874/46874 [==============================] - 8s 169us/step - loss: 0.1000 - val

46875/46875 [==============================] - 8s 179us/step - loss: 0.0963 - val_loss: 0.0940
Epoch 27/100
46875/46875 [==============================] - 8s 179us/step - loss: 0.0959 - val_loss: 0.0939
Epoch 28/100
46875/46875 [==============================] - 8s 179us/step - loss: 0.0956 - val_loss: 0.0946
Epoch 29/100
46875/46875 [==============================] - 8s 177us/step - loss: 0.0957 - val_loss: 0.0974
Epoch 30/100
46875/46875 [==============================] - 8s 173us/step - loss: 0.0954 - val_loss: 0.0947
Epoch 31/100
46875/46875 [==============================] - 8s 169us/step - loss: 0.0956 - val_loss: 0.0942
Epoch 32/100
46875/46875 [==============================] - 8s 168us/step - loss: 0.0953 - val_loss: 0.0939
Epoch 33/100
46875/46875 [==============================] - 8s 168us/step - loss: 0.0954 - val_loss: 0.0938
Epoch 34/100
46875/46875 [==============================] - 8s 168us/step - loss: 0.0950 - val_loss: 0.0937
Epoch 35/100
46875/46875 [===============

Epoch 56/100
46875/46875 [==============================] - 8s 169us/step - loss: 0.0957 - val_loss: 0.0959
Restoring model weights from the end of the best epoch
Epoch 00056: early stopping
Train on 46875 samples, validate on 20090 samples
Epoch 1/100
46875/46875 [==============================] - 11s 236us/step - loss: 0.1440 - val_loss: 0.1177
Epoch 2/100
46875/46875 [==============================] - 8s 170us/step - loss: 0.1159 - val_loss: 0.1100
Epoch 3/100
46875/46875 [==============================] - 8s 178us/step - loss: 0.1101 - val_loss: 0.1043
Epoch 4/100
46875/46875 [==============================] - 8s 173us/step - loss: 0.1071 - val_loss: 0.1050
Epoch 5/100
46875/46875 [==============================] - 8s 176us/step - loss: 0.1055 - val_loss: 0.1018
Epoch 6/100
46875/46875 [==============================] - 8s 175us/step - loss: 0.1047 - val_loss: 0.1031
Epoch 7/100
46875/46875 [==============================] - 8s 175us/step - loss: 0.1034 - val_loss: 0.1003
Epoch 8/1

* Check pipe
* Add L2 reg
* look for text embeddings
* preprocess text
* build archi
* print archi
* pretrained emb
* tqdm notebook

Content based : no interaction influencer/artist taken into account (no embedding for them)

advantage : cold start allowed
disadvantage : interesting info lossed

==> hybrid recommender system

label encoded influencer kind : in production, a category 'Other' can be created to account for potential kinds not present in current dataset.

Grid search